In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [36]:
import os
root_logdir = os.path.join(os.curdir, "model2_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [32]:
batch_size = 32
target_size = (128,96)

train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255, 
                                  validation_split=0.1)


train_generator = train_datagen.flow_from_directory('../data/images/train',
                                                    color_mode='rgb',
                                                    target_size=target_size,
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    subset='training')

validation_generator = test_datagen.flow_from_directory('../data/images/train',
                                                       color_mode='rgb',
                                                       target_size=target_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical',
                                                       subset='validation')


Found 2051 images belonging to 6 classes.
Found 225 images belonging to 6 classes.


In [33]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 6, strides=2, padding='valid', input_shape=[target_size[0],target_size[1],3]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='valid'),
    keras.layers.Conv2D(128, 5, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.3),
    keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='valid'),
    keras.layers.Conv2D(256, 3, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(128, 3, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(64, 3, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size=(3,3), strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(6, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 62, 46, 64)        6976      
_________________________________________________________________
batch_normalization_50 (Batc (None, 62, 46, 64)        256       
_________________________________________________________________
activation_50 (Activation)   (None, 62, 46, 64)        0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 30, 22, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 30, 22, 128)       204928    
_________________________________________________________________
batch_normalization_51 (Batc (None, 30, 22, 128)       512       
_________________________________________________________________
activation_51 (Activation)   (None, 30, 22, 128)     

In [37]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('model2.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir())


history = model.fit(
    train_generator,
    validation_data = validation_generator, 
    verbose=1,
    epochs = 50,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

Epoch 1/50
65/65 [==============================] - 66s 1s/step - loss: 1.5645 - accuracy: 0.3491 - val_loss: 1.8012 - val_accuracy: 0.1911
Epoch 2/50
65/65 [==============================] - 65s 992ms/step - loss: 1.4750 - accuracy: 0.4135 - val_loss: 2.0062 - val_accuracy: 0.1911
Epoch 3/50
65/65 [==============================] - 64s 978ms/step - loss: 1.4255 - accuracy: 0.4359 - val_loss: 1.9050 - val_accuracy: 0.2000
Epoch 4/50
65/65 [==============================] - 65s 993ms/step - loss: 1.4088 - accuracy: 0.4471 - val_loss: 1.6313 - val_accuracy: 0.3378
Epoch 5/50
65/65 [==============================] - 64s 982ms/step - loss: 1.3854 - accuracy: 0.4695 - val_loss: 1.6560 - val_accuracy: 0.2889
Epoch 6/50
65/65 [==============================] - 65s 997ms/step - loss: 1.3922 - accuracy: 0.4617 - val_loss: 1.8707 - val_accuracy: 0.2356
Epoch 7/50
65/65 [==============================] - 65s 996ms/step - loss: 1.4046 - accuracy: 0.4627 - val_loss: 1.8034 - val_accuracy: 0.2711
Ep

KeyboardInterrupt: 